# Nanosafety Data Interface Survey

Nanosafety data interface – online user interface enabling user friendly access to the aggregated search index of (sub)set of eNanoMapper database instances. Usually the user interface is project specific and protected but can be also publicly available. Multiple project specific interfaces at https://search.data.enanomapper.net/.


In [31]:
import yaml
from pynanomapper import aa
from pynanomapper import units
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from pynanomapper import client_solr
from pynanomapper import client_ambit
from pynanomapper import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import importlib
import warnings
import qgrid 

import matplotlib.pyplot as plt
import io
from measurement.utils import guess
from measurement.measures import Distance, Temperature, Volume, Area, Frequency, Mass, Time
#init_notebook_mode(connected=True)
warnings.simplefilter("ignore")

import logging,os,time
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')
logger = logging.getLogger()
logger.info('Started at %s \t%s',os.name, time.time())

2020-06-27 13:39:10,643  INFO     Started at nt 	1593254350.6420085


In [32]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

Select enanoMapper aggregated search service:
Enter `X-Gravitee-Api-Key` you have received upon subscription to http://api.ideaconsult.net


interactive(children=(Dropdown(description='Service:', options=('https://api.ideaconsult.net/enanomapper', 'ht…

In [37]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))

Sending queries to https://api.ideaconsult.net/nanoreg1


In [40]:
def log_query(q):
    #logger.debug(q)
    pass
    
def beautify(r):
    pass
facets = client_solr.Facets()    

# Database records

In [41]:
projects = ["NanoReg2","OMICS_DATA","NANoREG","NanoTest","ENPRA","MARINA","NanoGenotox","SANOWORK","caNanoLab","eNanoMapper"]
for project in projects:
    dfm = facets.summary(service_uri,auth_object, query="owner_name_hs:{}".format(project),fields=["owner_name_hs","substanceType_hs","s_uuid_hs","publicname_hs","name_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    

    if dfm.shape[0]>0:
        dfm.to_csv("materials_{}.csv".format(project))  
        dfs = facets.summary(service_uri,auth_object, query="owner_name_s:{}".format(project),fields=["owner_name_s","document_uuid_s"],fq='type_s:study',log_query=log_query,log_result=beautify)    

        dfs.to_csv("study_{}.csv".format(project))
        print("Project `{}` Materials {} Studies {} Data {}".format(project,dfm.shape[0],dfs.shape[0],dfs["Number of data points"].sum()))


Project `NANoREG` Materials 147 Studies 5144 Data 39196


# Endpoints

In [42]:

facets = client_solr.Facets()
df = facets.summary(service_uri,auth_object, query="*:*",fields=["topcategory_s","endpointcategory_s"],log_query=log_query,log_result=beautify)    

df.rename(columns={'owner_name_s':'project','endpointcategory_name' : 'endpoint', 'topcategory_s' : 'category'},inplace=True)
df.to_csv("endpoints_all.csv")  
qgrid.show_grid(df[["category","endpoint","Number of data points"]])


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [43]:
facets = client_solr.Facets()
df = facets.summary(service_uri,auth_object, query="*:*",fields=["topcategory_s","endpointcategory_s","owner_name_s"],log_query=log_query,log_result=beautify)    

df.rename(columns={'owner_name_s':'project','endpointcategory_name' : 'endpoint', 'topcategory_s' : 'category'},inplace=True)

In [44]:
qgrid.show_grid(df[["project","category","endpoint","Number of data points"]])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Material types

In [45]:

for project in projects:
    dfm = facets.summary(service_uri,auth_object, query="owner_name_hs:{}".format(project),fields=["owner_name_hs","substanceType_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    
    if dfm.shape[0]>0:
        dfm.to_csv("materialtypes_{}.csv".format(project))  
        print("Project `{}` Material types {}".format(project,dfm.shape[0]))
        #display(dfm)

Project `NANoREG` Material types 25


In [48]:
# All material types
dfm = facets.summary(service_uri,auth_object, query="*:*".format(project),fields=["substanceType_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    
dfm.to_csv("materialtypes_all.csv".format(project))  
dfm    

,substanceType_hs,Number of data points,substanceType_name
0,NPO_1373,38,SiO2 nanoparticle
1,NPO_354,30,MWCNT
2,CHEBI_133349,12,nanofibrillar cellulose
3,NPO_1486,11,TiO2 nanoparticle
4,NPO_1892,8,Ag nanoparticle
5,CHEBI_3311,7,CaCO3
6,CHEBI_51050,7,TiO2 nanoparticle
7,NPO_1542,7,ZnO nanoparticle
8,ENM_9000006,5,CeO nanoparticle
9,CHEBI_59999,3,chemical substance
